<a href="https://colab.research.google.com/github/rufous86/hh_api/blob/main/hh_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import plotly.express as px

In [70]:
res = []
def get_vacancy(vacancies, pages=30):
    for vacancy in vacancies:
        print(f'collecting {vacancy}')
        for page in tqdm(range(pages)):
            params = {
                'text': f'NAME:{vacancy}',
                'page': page,
                'per_page': 100
                }
            req = requests.get('https://api.hh.ru/vacancies', params).json()
            if 'items' in req.keys():
                res.extend(req['items'])

vacancies = ['аналитик', 'данные', 'data', 'analyst']
get_vacancy(vacancies)

data = pd.DataFrame(res)
data.shape

collecting аналитик


  0%|          | 0/30 [00:00<?, ?it/s]

collecting данные


  0%|          | 0/30 [00:00<?, ?it/s]

collecting data


  0%|          | 0/30 [00:00<?, ?it/s]

collecting analyst


  0%|          | 0/30 [00:00<?, ?it/s]

(5074, 30)

In [71]:
df = data.copy()
columns = ['name', 'salary', 'schedule', 'area', 'address']
df = df[columns]
df.head()

,name,salary,schedule,area,address
0,Системный аналитик\Бизнес аналитик,"{'from': 150000, 'to': 250000, 'currency': 'RU...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'city': 'Санкт-Петербург', 'street': 'улица М..."
1,Аналитик,"{'from': 200000, 'to': 200000, 'currency': 'KZ...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '160', 'name': 'Алматы', 'url': 'https:...","{'city': 'Алматы', 'street': 'улица Сатпаева',..."
2,Аналитик,None,"{'id': 'fullDay', 'name': 'Полный день'}","{'id': '1', 'name': 'Москва', 'url': 'https://...","{'city': 'Москва', 'street': 'улица Обручева',..."
3,Бизнес-аналитик (банковское направление),"{'from': 250000, 'to': 300000, 'currency': 'RU...","{'id': 'flexible', 'name': 'Гибкий график'}","{'id': '1', 'name': 'Москва', 'url': 'https://...","{'city': 'Москва', 'street': None, 'building':..."
4,Аналитик,None,"{'id': 'fullDay', 'name': 'Полный день'}","{'id': '41', 'name': 'Калининград', 'url': 'ht...","{'city': 'посёлок Поддубное', 'street': 'Лесна..."


In [72]:
df["city"] = (df["area"]
                     .apply(lambda x: x.get("name") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["schedule"] = (df["schedule"]
                     .apply(lambda x: x.get("id") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_from"] = (df["salary"]
                     .apply(lambda x: x.get("from") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_to"] = (df["salary"]
                     .apply(lambda x: x.get("to") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_currency"] = (df["salary"]
                     .apply(lambda x: x.get("currency") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["lat"] = (df["address"]
                     .apply(lambda x: x.get("lat") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["lon"] = (df["address"]
                     .apply(lambda x: x.get("lng") 
                                      if isinstance(x, dict) 
                                      else np.nan))

In [73]:
df.head()

,name,salary,schedule,area,address,city,salary_from,salary_to,salary_currency,lat,lon
0,Системный аналитик\Бизнес аналитик,"{'from': 150000, 'to': 250000, 'currency': 'RU...",fullDay,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'city': 'Санкт-Петербург', 'street': 'улица М...",Санкт-Петербург,150000.0,250000.0,RUR,59.888803,30.279828
1,Аналитик,"{'from': 200000, 'to': 200000, 'currency': 'KZ...",fullDay,"{'id': '160', 'name': 'Алматы', 'url': 'https:...","{'city': 'Алматы', 'street': 'улица Сатпаева',...",Алматы,200000.0,200000.0,KZT,43.236861,76.911248
2,Аналитик,None,fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'city': 'Москва', 'street': 'улица Обручева',...",Москва,NaN,NaN,NaN,55.656189,37.526169
3,Бизнес-аналитик (банковское направление),"{'from': 250000, 'to': 300000, 'currency': 'RU...",flexible,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'city': 'Москва', 'street': None, 'building':...",Москва,250000.0,300000.0,RUR,55.736146,37.595016
4,Аналитик,None,fullDay,"{'id': '41', 'name': 'Калининград', 'url': 'ht...","{'city': 'посёлок Поддубное', 'street': 'Лесна...",Калининград,NaN,NaN,NaN,54.609839,20.481157


In [74]:
df.tail()

,name,salary,schedule,area,address,city,salary_from,salary_to,salary_currency,lat,lon
5069,Аналитик 1С,"{'from': 200000, 'to': None, 'currency': 'RUR'...",fullDay,"{'id': '47', 'name': 'Кемерово', 'url': 'https...",None,Кемерово,200000.0,NaN,RUR,NaN,NaN
5070,Инвестиционный аналитик / консультант,"{'from': 200000, 'to': 350000, 'currency': 'RU...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,Москва,200000.0,350000.0,RUR,NaN,NaN
5071,Главный аналитик (Руководитель IT проектов),"{'from': None, 'to': 230000, 'currency': 'RUR'...",fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'city': 'Москва', 'street': 'Днепропетровская...",Москва,NaN,230000.0,RUR,55.623117,37.60399
5072,Аналитик по расследованию компьютерных инцидентов,None,fullDay,"{'id': '68', 'name': 'Омск', 'url': 'https://a...",None,Омск,NaN,NaN,NaN,NaN,NaN
5073,Системный аналитик (госуслуги),None,fullDay,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,Москва,NaN,NaN,NaN,NaN,NaN


In [75]:
df['salary_currency'].value_counts()

RUR    1273
USD      47
KZT      33
BYR      13
EUR      13
UZS      12
KGS       1
AZN       1
Name: salary_currency, dtype: int64

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5074 entries, 0 to 5073
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             5074 non-null   object 
 1   salary           1393 non-null   object 
 2   schedule         5074 non-null   object 
 3   area             5074 non-null   object 
 4   address          2509 non-null   object 
 5   city             5074 non-null   object 
 6   salary_from      1194 non-null   float64
 7   salary_to        808 non-null    float64
 8   salary_currency  1393 non-null   object 
 9   lat              2356 non-null   float64
 10  lon              2356 non-null   float64
dtypes: float64(4), object(7)
memory usage: 436.2+ KB


In [77]:
df = df[df['salary_currency'] == 'RUR'][['name', 'schedule', 'city', 'salary_from', 'salary_to', 'salary_currency', 'lat', 'lon']]

In [78]:
df = df.dropna(axis=0, subset=['lat', 'lon'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742 entries, 0 to 5071
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             742 non-null    object 
 1   schedule         742 non-null    object 
 2   city             742 non-null    object 
 3   salary_from      637 non-null    float64
 4   salary_to        414 non-null    float64
 5   salary_currency  742 non-null    object 
 6   lat              742 non-null    float64
 7   lon              742 non-null    float64
dtypes: float64(4), object(4)
memory usage: 52.2+ KB


In [79]:
df['salary_from'].min(), df['salary_from'].max()

(10000.0, 350000.0)

In [80]:
df['salary_to'].min(), df['salary_to'].max()

(160.0, 450000.0)

In [81]:
df[df['salary_to'] == 160]

,name,schedule,city,salary_from,salary_to,salary_currency,lat,lon
333,Системный аналитик,flexible,Москва,NaN,160.0,RUR,55.66076,37.54356


In [84]:
link = "https://raw.githubusercontent.com/timurkanaz/Russia_geojson_OSM/master/GeoJson's/Countries/Russia_regions.geojson"

In [111]:
from urllib.request import urlopen
import json
with urlopen(link) as response:
    counties = json.load(response)

# counties["features"][0]

In [109]:
fig = px.scatter_geo(df, lat='lat', lon='lon', text='city', width=1600, height=1600)
fig.show()